# Factory

It allow to create objects in a uniformed manner without considering the detail of their implementations details.
It is used normally:
 - when you cannot track the objects created by your application because the code that creates them is in many different places instead of a single function.

The main components:
 - Products: The products classes from what we want to create objects.
 - Creator: It declares the Factory Method, which is essentially a method for creating objects.


## Factory Method

This is used when the products are based on the same classes. So the factory can be used to create object according to certain conditions (example 1) or operations (example 2).

In [21]:
# example 1
class Character:
    def __init__(self, name, attack, health):
        self.name = name
        self.attack = attack
        self.health = health

class CharacterFacotry:
    @staticmethod
    def create_player(name, attack, health):
        return Character(name, attack, health)
    @staticmethod
    def create_weapon(name, attack):
        return Character("weapon_"+name, attack, 0)
    @staticmethod
    def create_npc(name):
        return Character("npc_"+name, 0, 0)

In [22]:
player = CharacterFacotry.create_player("ninja", 100, 100)
weapon = CharacterFacotry.create_weapon("knife", 80)
cat = CharacterFacotry.create_npc("cat")

In [3]:
# example 2
from math import *
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class PointFactory:
    @staticmethod
    def create_cartesian(x, y):
        return Point(x, y)
    @staticmethod
    def create_polar(r, theta):
        return Point(r*cos(theta), r*sin(theta))

In [4]:
p1 = PointFactory.create_cartesian(3, 8)
p2 = PointFactory.create_polar(4, 1.2)

## Inner Factory

The advantage of this is the factory class has access to the product class's attributes.

In [6]:
class Character:
    def __init__(self, name, attack, health):
        self.name = name
        self.attack = attack
        self.health = health
        self.factory = self.CharacterFacotry

    class CharacterFacotry:
        @staticmethod
        def create_player(name, attack, health):
            return Character(name, attack, health)
        @staticmethod
        def create_weapon(name, attack):
            return Character("weapon_"+name, attack, 0)
        @staticmethod
        def create_npc(name):
            return Character("npc_"+name, 0, 0)

In [7]:
player = Character.CharacterFacotry.create_player("ninja", 100, 100)
weapon = Character.CharacterFacotry.create_weapon("knife", 80)
cat = Character.CharacterFacotry.create_npc("cat")

## Factory

This is a more common example where we use factory where there are several product classes.

In [19]:
import abc

# product classes
class Character(abc.ABC):
    def __init__(self, name):
        self.name = name

    @abc.abstractmethod
    def show_info(self):
        raise NotImplementedError

class Player(Character):
    def __init__(self, name):
        super().__init__(name)
        
    def show_info(self):
        print(f"created player: {self.name}")

class Enemy(Character):
    def __init__(self, name):
        super().__init__(name)

    def show_info(self):
        print(f"created enemy: {self.name}")

class Passerby(Character):
    def __init__(self, name):
        super().__init__(name)

    def show_info(self):
        print(f"created passby: {self.name}")

# factory class
class CharacterFactory:
    @staticmethod
    def get_character(ctype=None):
        if ctype is None:
            return None
        elif ctype == "player":
            return Player
        elif ctype == "ennemy":
            return Ennemy
        elif ctype == "passerby":
            return Passerby
        else:
            return None

In [20]:
player = CharacterFactory.get_character("player")
player = player("ninja")
player.show_info()

created player: ninja


# Using a Registry

In cases where product classes are all over the places, it is more secure to use a registry to record the product classes than using if statements like above. 
This way is more scalable and safer when we add other classes.

In [2]:
class Registry:
    REGISTRY = {}
    
    @classmethod
    def register_character(self, type_name):
        def wrapper(cls):
            self.REGISTRY[type_name] = cls
            return cls
        return wrapper

    @classmethod
    def get_character(self, type_name):
        return self.REGISTRY[type_name]
        
@Registry.register_character("player")
class Player(Character):
    def __init__(self, name):
        super().__init__(name)
    def show_info(self):
        print(f"created player: {self.name}")
        
@Registry.register_character("ennemy")
class Ennemy(Character):
    def __init__(self, name):
        super().__init__(name)
    def show_info(self):
        print(f"created ennemy: {self.name}")
        
@Registry.register_character("passerby")
class Passerby(Character):
    def __init__(self, name):
        super().__init__(name)
    def show_info(self):
        print(f"created passerby: {self.name}")

In [36]:
player = Registry.get_character("player")
player = player("ninja")
player.show_info()

ennemy = Registry.get_character("ennemy")
ennemy = ennemy("bat")
ennemy.show_info()

passerby = Registry.get_character("passerby")
passerby = passerby("cat")
passerby.show_info()

created player: ninja
created ennemy: bat
created passerby: cat


# Abstract Factory

In [31]:
class EntityRegistry:
    REGISTRY = {}
    
    @classmethod
    def register_entity(self, type_name):
        def wrapper(cls):
            self.REGISTRY[type_name] = cls
            return cls
        return wrapper

    @classmethod
    def get_entity(self, type_name):
        return self.REGISTRY[type_name]

In [43]:
# example of a game engine

import abc

class Object(abc.ABC):
    """
    Entities to be displayed in game.
    """
    pass


class Character(Object):
    """
    Entities being part of the game AI.
    """
    @abc.abstractmethod
    def draw(self):
        pass

@EntityRegistry.register_entity("player")
class Player(Character):
    def __init__(self, name):
        self.name = name
        self.characteristic = {"health": 100, "attack": 100, "intelligence": 100}
        self.texture = {"run": "run_text", "jump": "jump_tex", "walk": "walk_tex"}
        self.position = (0, 0)
    def prepare(self):
        print(f"Init player {self.name} -> set characteristics -> load texture")
    def draw(self):
        print(f"draw player {self.name}")

@EntityRegistry.register_entity("enemy")
class Enemy(Character):
    def __init__(self, name):
        self.name = name
        self.characteristic = {"health": 100, "attack": 100, "intelligence": 40}
        self.texture = {"run": "run_text", "jump": "jump_tex", "walk": "walk_tex"}
        self.position = (0, 0)
    def prepare(self):
        print(f"Init Enemy {self.name} -> set characteristics -> load texture -> set AI")
    def draw(self):
        print(f"draw Enemy {self.name}")

@EntityRegistry.register_entity("npc")
class NPC(Character):
    def __init__(self, name):
        self.name = name
        self.texture = {"run": "run_text", "jump": "jump_tex", "walk": "walk_tex"}
        self.position = (0, 0)
    def prepare(self):
        print(f"Init NPC {self.name} -> load texture")
    def draw(self):
        print(f"draw NPC {self.name}")

class SceneObject(Object):
    """
    Static entities for scene.
    """
    @abc.abstractmethod
    def draw(self):
        pass

@EntityRegistry.register_entity("tile")
class tiles(SceneObject):
    def __init__(self):
        self.tileinfo = {"tilenum": 0, "tilefile": "path_to_tilefile"}
        self.position = (0, 0)
    def prepare(self):
        print(f"Init tile num {self.tileinfo["tilenum"]} -> load texture")
    def draw(self):
        print(f"draw tile {self.tileinfo["tilenum"]}")

@EntityRegistry.register_entity("background")
class BackgroundObject(Object):
    """
    Background of the scene.
    """
    def prepare(self):
        print(f"Init background image -> load image")
    def draw(self):
        pass

In [45]:
# The Abstract Factory
class GameElementFactory:
    def make_characters(self): pass
    def make_scene(self): pass
    def make_background(self): pass

# concrete factory
class Level1Factory(GameElementFactory):
    def make_characters(self):
        self.player = EntityRegistry.get_entity("player")("ninja")
        self.player.prepare()
        self.player.draw()
        self.enemy = EntityRegistry.get_entity("enemy")("monster")
        self.enemy.prepare()
        self.enemy.draw()
        self.npc = EntityRegistry.get_entity("npc")("cat")
        self.npc.prepare()
        self.npc.draw()
    def make_scene(self): 
        self.tile0 = EntityRegistry.get_entity("tile")()
        self.tile0.prepare()
        self.tile0.draw()
    def make_background(self): 
        self.bg0 = EntityRegistry.get_entity("background")()
        self.bg0.prepare()
        self.bg0.draw()

class Level2Factory(GameElementFactory):
    def make_characters(self):
        self.player = EntityRegistry.get_entity("player")("mario")
        self.player.prepare()
        self.player.draw()
        self.enemy = EntityRegistry.get_entity("enemy")("frog")
        self.enemy.prepare()
        self.enemy.draw()
    def make_scene(self): 
        self.tile0 = EntityRegistry.get_entity("tile")()
        self.tile0.prepare()
        self.tile0.draw()
    def make_background(self): 
        self.bg0 = EntityRegistry.get_entity("background")()
        self.bg0.prepare()
        self.bg0.draw()

class Game:
    def __init__(self, factory):
        self.factory = factory
        player = factory.make_characters()
        enemy = factory.make_scene()
        npc = factory.make_background()
    def play(self):
        print("Game start.")

In [46]:
level1 = Game(Level1Factory())
level2 = Game(Level2Factory())

Init player ninja -> set characteristics -> load texture
draw player ninja
Init Enemy monster -> set characteristics -> load texture -> set AI
draw Enemy monster
Init NPC cat -> load texture
draw NPC cat
Init tile num 0 -> load texture
draw tile 0
Init background image -> load image
Init player mario -> set characteristics -> load texture
draw player mario
Init Enemy frog -> set characteristics -> load texture -> set AI
draw Enemy frog
Init tile num 0 -> load texture
draw tile 0
Init background image -> load image


## ref
https://python-3-patterns-idioms-test.readthedocs.io/en/latest/Factory.html